In [1]:
from elasticsearch import Elasticsearch, RequestsHttpConnection

In [2]:
# connect elasticsearch cluster
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
e_dict = {'e1' : {
    "movie":"Toy Story",
    "country": "USA, Canada",
    "genre": ["Animation","Children's","Comedy"],
    "year_release": "1995",
    "auther": "john",
    "describe": "the love story of Linda and Jub",
    "star": 2,
    },
   'e2' : {
    "movie":"Titanic",
    "country": "USA, England",
    "genre": ["Children's","drama"],
    "year_release": "2000",
    "auther": "kevin",
    "describe": "the journey on the atlantic ocean of Titanic ship",
    "star": 5,
    },
    'e3' : {
    "movie":"Iron Man",
    "country": "USA",
    "genre": ["drama","War"],
    "year_release": "2010",
    "auther": "Max",
    "describe": "the story talk about Tony stark, a super hero",
    "star": 3,
    },
    'e4' : {
    "movie":"Naruto",
    "country": "Japan",
    "genre": ["Animation","War"],
    "year_release": "1998",
    "auther": "Kimito",
    "describe": "talk the journe to become a super ninja of Naruto",
    "star": 4,
    },      }

# insert

In [4]:
# let's store this document in elasticsearch
for i,j in zip(e_dict, range(1, len(e_dict)+1)):
    print(i)
    res = es.index(index = 'idmov', doc_type = 'movie', id = j, body = e_dict[i])
    print(res['result'])

e1
updated
e2
updated
e3
updated
e4
updated


# retrieving the document in elasticsearch use : get

In [5]:
# retrieving a document in elasticsearch
a = es.get(index = 'idmov', doc_type= 'movie', id = 1)
a

{'_id': '1',
 '_index': 'idmov',
 '_source': {'auther': 'john',
  'country': 'USA, Canada',
  'describe': 'the love story of Linda and Jub',
  'genre': ['Animation', "Children's", 'Comedy'],
  'movie': 'Toy Story',
  'star': 2,
  'year_release': '1995'},
 '_type': 'movie',
 '_version': 8,
 'found': True}

In [6]:
a['_id']

'1'

In [7]:
# retrieving the documents in elasticsearch
m = dict()
for i,j in zip(e_dict,range(1, len(e_dict)+ 1)):
    m[i] = es.get(index = 'idmov', doc_type = 'movie', id = int(j))

In [8]:
#m

# search the document in elasticsearch from the content of the filed in document

# using query statement: "Match Query"
+ the standard query for performing full text queries, including fuzzy matching and phrase or proximity queries

In [9]:
# match query request accept text/numerics/dates, analyzer them
a1 = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match": {
            "genre" : "ani war"
        }
    }
})
a1['hits']['total']

2

In [10]:
a1

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'idmov',
    '_score': 0.6931472,
    '_source': {'auther': 'Kimito',
     'country': 'Japan',
     'describe': 'talk the journe to become a super ninja of Naruto',
     'genre': ['Animation', 'War'],
     'movie': 'Naruto',
     'star': 4,
     'year_release': '1998'},
    '_type': 'movie'},
   {'_id': '3',
    '_index': 'idmov',
    '_score': 0.2876821,
    '_source': {'auther': 'Max',
     'country': 'USA',
     'describe': 'the story talk about Tony stark, a super hero',
     'genre': ['drama', 'War'],
     'movie': 'Iron Man',
     'star': 3,
     'year_release': '2010'},
    '_type': 'movie'}],
  'max_score': 0.6931472,
  'total': 2},
 'timed_out': False,
 'took': 1038}

In [11]:
print(a1['hits']['total'])
print(a1['hits']['max_score'])

2
0.6931472


In [12]:
a1['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 0.6931472,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [13]:
# take the document has the result most proximity

In [14]:
# the way 1
a1['hits']['hits'][0]

{'_id': '4',
 '_index': 'idmov',
 '_score': 0.6931472,
 '_source': {'auther': 'Kimito',
  'country': 'Japan',
  'describe': 'talk the journe to become a super ninja of Naruto',
  'genre': ['Animation', 'War'],
  'movie': 'Naruto',
  'star': 4,
  'year_release': '1998'},
 '_type': 'movie'}

In [15]:
a1['hits']['hits'][0]['_id']

'4'

In [16]:
# the way 2 : search the document has the reusult most proximity
max_score = a1['hits']['max_score']
for i in a1['hits']['hits']:
    if(i['_score'] == max_score):
        print(i)

{'_index': 'idmov', '_type': 'movie', '_id': '4', '_score': 0.6931472, '_source': {'movie': 'Naruto', 'country': 'Japan', 'genre': ['Animation', 'War'], 'year_release': '1998', 'auther': 'Kimito', 'describe': 'talk the journe to become a super ninja of Naruto', 'star': 4}}


Match

The match query is of type boolean. It means that the text provided is analyzed and the analysis process constructs a boolean query from the provided text. The operator flag can be set to or or and to control the boolean clauses (defaults to or). The minimum number of optional should clauses to match can be set using the minimum_should_match parameter.

The analyzer can be set to control which analyzer will perform the analysis process on the text. It defaults to the field explicit mapping definition, or the default search analyzer.

The lenient parameter can be set to true to ignore exceptions caused by data-type mismatches, such as trying to query a numeric field with a text query string. Defaults to false.

# using query Statement : "fuzzy search"
+ fuzzy search use Lavenshtein Edit Distance to allow config parameter fuzziness is gave the fit reusult for need

In [17]:
# using operator parameter
a2 = es.search(index = 'idmov', doc_type = "movie", body = {
    "query": {
        "match" : {
            "describe" : {
                "query" : "the story ton",
                "operator" : "or"
            }
        }
    }
})
a2['hits']['total']

4

In [18]:
a2 = es.search(index = 'idmov', doc_type = "movie", body = {
    "query": {
        "match" : {
            "describe" : {
                "query" : "the story ton",
                "operator" : "and"
            }
        }
    }
})
a2['hits']['total']

0

In [19]:
# match query
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match": {
            "describe": "journey to"
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 0.7083998,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '4',
  '_index': 'idmov',
  '_score': 0.6785375,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

In [20]:
# match query
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match": {
            "describe": "journ to"
        }
    }
})
a['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 0.6785375,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

In [21]:
# match query
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match": {
            "describe": "becom to"
        }
    }
})
a['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 0.6785375,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

In [22]:
# phrase_match
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match_phrase": {
            "describe": "the journey"
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 0.89473325,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'}]

In [23]:
# phrase_match
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match_phrase": {
            "describe": "the journe"
        }
    }
})
a['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 0.8570162,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

In [24]:
# phrase_match : search exact documnent contain "the bec" of the field "describe", result : empty
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match_phrase": {
            "describe": "the bec"
        }
    }
})
a['hits']['hits']

[]

In [25]:
# prefix query
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "prefix": {
            "genre": "dram"
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [26]:
# prefix query : we is received empty, baucause prefix query search the field "genre" contain "dram a"
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "prefix": {
            "genre": "dram a"
        }
    }
})
a['hits']['hits']

[]

In [27]:
# term query finds document that contain exact term specified in the inverted index
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "term": {
            "genre": "drama"
        }
    }
    })
a

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2',
    '_index': 'idmov',
    '_score': 0.6931472,
    '_source': {'auther': 'kevin',
     'country': 'USA, England',
     'describe': 'the journey on the atlantic ocean of Titanic ship',
     'genre': ["Children's", 'drama'],
     'movie': 'Titanic',
     'star': 5,
     'year_release': '2000'},
    '_type': 'movie'},
   {'_id': '3',
    '_index': 'idmov',
    '_score': 0.2876821,
    '_source': {'auther': 'Max',
     'country': 'USA',
     'describe': 'the story talk about Tony stark, a super hero',
     'genre': ['drama', 'War'],
     'movie': 'Iron Man',
     'star': 3,
     'year_release': '2010'},
    '_type': 'movie'}],
  'max_score': 0.6931472,
  'total': 2},
 'timed_out': False,
 'took': 2}

In [28]:
# term query: result empty, because term query not finds any content "dram" in the field "genre" of the documents
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "term": {
            "genre": "dram"
        }
    }
    })
a

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [], 'max_score': None, 'total': 0},
 'timed_out': False,
 'took': 25}

In [29]:
# terms query 
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "terms": {
            "genre":[ "drama", 'war']
        }
    }
    })
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '4',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [30]:
# terms query 
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "terms": {
            "genre":[ "dram a", 'animation']
        }
    }
    })
a['hits'],[ 'hits']

({'hits': [{'_id': '4',
    '_index': 'idmov',
    '_score': 1.0,
    '_source': {'auther': 'Kimito',
     'country': 'Japan',
     'describe': 'talk the journe to become a super ninja of Naruto',
     'genre': ['Animation', 'War'],
     'movie': 'Naruto',
     'star': 4,
     'year_release': '1998'},
    '_type': 'movie'},
   {'_id': '1',
    '_index': 'idmov',
    '_score': 1.0,
    '_source': {'auther': 'john',
     'country': 'USA, Canada',
     'describe': 'the love story of Linda and Jub',
     'genre': ['Animation', "Children's", 'Comedy'],
     'movie': 'Toy Story',
     'star': 2,
     'year_release': '1995'},
    '_type': 'movie'}],
  'max_score': 1.0,
  'total': 2},
 ['hits'])

In [31]:
# multi_match query 
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "multi_match": {
            "query": "becom to",
            "fields": ["describe", "author"]
        }
    }
})
a['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 0.6785375,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

In [32]:
# controlling percision : multi_match
#elasticsearch support :"minimum_should_match" parameter, we give out the term , from this term, we query to document
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "multi_match": {
            "query": "talk is journe ninja",
            # we will make query to the documents contain less 75% words in the phrases " talk the journe to"
            "minimum_should_match": "75%"
        }
    }
})
a['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 2.0356126,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

# bool query
+ a query that matches documents matching boolen combinations of other queries
+ queries : must, not must, filter, should

In [33]:
# must query : the clause(query) appear in the matching document
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool": {
            "must": {
                "term" : {"auther": "joh"}
            }
        }
    }
})
a['hits']['hits']

[]

In [34]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool": {
            "must": {
                "term" : {"auther": "john"}
            }
        }
    }
})
a['hits']['hits']

[{'_id': '1',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'john',
   'country': 'USA, Canada',
   'describe': 'the love story of Linda and Jub',
   'genre': ['Animation', "Children's", 'Comedy'],
   'movie': 'Toy Story',
   'star': 2,
   'year_release': '1995'},
  '_type': 'movie'}]

In [35]:
# must_not query : the clause (query) must not appear in the matching documents
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool": {
            "must_not": {
                "term" : {"auther": "john"}
            }
        }
    }
})
a['hits']['total']

3

In [36]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool": {
            "must_not": {
                "range" : {
                    "star": {"lte":3,"gte": 5}
                }
            }
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '4',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'},
 {'_id': '1',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'john',
   'country': 'USA, Canada',
   'describe': 'the love story of Linda and Jub',
   'genre': ['Animation', "Children's", 'Comedy'],
   'movie': 'Toy Story',
   'star': 2,
   'year_release': '1995'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Max',
   'country':

In [37]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "must" : {
            "term" : { "genre" : "drama" }
          },
          "must_not" : {
              "term" : { "auther": "kevin" }
                }
          }
        }
})
a['hits']['hits']

[{'_id': '3',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [38]:
# combining queries : bool query (must - must_not)
#we can make query data form the documents with the many terms 
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool": {
            "must": {"match": {"genre" : "drama"}},
            "must_not": {"match": {"auther": "kevin"}}
        }
    }
})
a['hits']['hits']

[{'_id': '3',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [39]:
# filter query : the clause (query) must appear in the matching document
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "filter" : {
            "term" : { "describe" : "the story" }
          }
          }
        }
})
a['hits']['hits']

[]

In [40]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "filter" : {
            "term" : { "genre" : "drama" }
          }
          }
        }
})
a['hits']['total']

2

In [41]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "must" : {
            "term" : { "genre" : "drama" }
          },
          "filter": {
              "term": {"star": 3}
          },
          "must_not" : {
              "term" : { "auther": "kevin" }
                }
          }
        }
})
a['hits']['hits']

[{'_id': '3',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [42]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "must" : {
            "term" : { "genre" : "drama" }
          },
          "filter": {
              "term": {"star": 5}
          },
          "must_not" : {
              "term" : { "auther": "kevin" }
                }
          }
        }
})
a['hits']['hits']

[]

In [43]:
# should query : the clause (query) appear in the matching document
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query":{
        "bool": {
            "should": {
                "term": {"auther": "kevin"}
            }
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 0.6931472,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'}]

In [44]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query":{
        "bool": {
            "should": [
                {"term": {"auther": "john"}},
                {"term": {"auther": "max"}}
            ],
            "minimum_should_match": 1,
            "boost": 1.0
        }
    }
})
a['hits']['total']

2

In [45]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "must" : {
            "term" : { "genre" : "drama" }
          },
          "filter": {
              "term": {"star": 3}
          },
          "should": {
              "term":{"country": "USA"}
          },
          "must_not" : {
              "term" : { "auther": "kevin" }
                }
          }
        }
})
a['hits']['hits']

[{'_id': '3',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [46]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "must" : {
            "term" : { "genre" : "drama" }
          },
          "filter": {
              "term": {"star": 3}
          },
          "should": {
              "term":{"country": "Japan"}
          },
          "must_not" : {
              "term" : { "auther": "kevin" }
                }
          }
        }
})
a['hits']['hits']

[{'_id': '3',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [47]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "bool" : {
          "must" : {
            "term" : { "genre" : "drama" }
          },
          "filter": {
              "term": {"star": 3}
          },
          "must_not" : {
              "term" : { "auther": "kevin" }
                },
          "should": [
                {"term": {"auther": "john"}},
                {"term": {"auther": "max"}}
            ],
            "minimum_should_match": 1,
            "boost": 1.0
          }
        }
})
a['hits']['hits']

[{'_id': '3',
  '_index': 'idmov',
  '_score': 0.5753642,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

# range query 
+ matches documents with fields that have terms within a certain range . 
+ num/ date fields

In [48]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "range": {
            "star" :{ "gt": 4, "lt": 5}
        }
    }
})
a['hits']['hits']

[]

In [49]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "range": {
            "star" :{ "gte": 3, "lte": 5}
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '4',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [50]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "range": {
            "star" :{ "gte": 4, "lte": 5, "boost": 2.0}
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 2.0,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '4',
  '_index': 'idmov',
  '_score': 2.0,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

# Query string
+ query string provide the queries : multi_match, bool, boosting, fuzzy matching, wildcards, regexp, range queries


In [51]:
a2 = es.search(index = 'idmov', doc_type = "movie", body = {
    "query":{
        "query_string":{
            "default_field": "describe",
            "query": "the jour of the Tony"
        }
    }
})
a2['hits']['total']

4

In [52]:
# controlling analysis : query string query
# Analyzer : is used the document standard, and make :
# character filters: preprocessing input data
# tokenizer: then string is cleared by "character filter", the make filter of tokenizer (we is define)
# tokenizer filter: the each term go thought tokenizer filter, the will clear (ex: remove the html characters , covert character
a2 = es.search(index = 'idmov', doc_type = "movie", body = {
    "query":{
        "query_string":{
            "default_field": "describe",
            "query": "journe is nin"
        }
    }
})
a2['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 0.6785375,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'}]

In [53]:
a2 = es.search(index = 'idmov', doc_type = "movie", body = {
    "query":{
        "query_string":{
            "default_field": "describe",
            "query": "journe is stark"
        }
    }
})
a2['hits']['hits']

[{'_id': '4',
  '_index': 'idmov',
  '_score': 0.6785375,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 0.2876821,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

# Zero term query

In [54]:
a_3 = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match" : {
            "describe" : {
                "query" : "the journe of",
                "operator" : "and",
                "zero_terms_query": "all"
            }
        }
    }
})

In [55]:
a_3

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'idmov',
    '_score': 1.0354949,
    '_source': {'auther': 'Kimito',
     'country': 'Japan',
     'describe': 'talk the journe to become a super ninja of Naruto',
     'genre': ['Animation', 'War'],
     'movie': 'Naruto',
     'star': 4,
     'year_release': '1998'},
    '_type': 'movie'}],
  'max_score': 1.0354949,
  'total': 1},
 'timed_out': False,
 'took': 6}

In [56]:
a_3 = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match" : {
            "describe" : {
                "query" : "the journe of",
                "operator" : "or",
                "zero_terms_query": "all"
            }
        }
    }
})
a_3

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'idmov',
    '_score': 1.0354949,
    '_source': {'auther': 'Kimito',
     'country': 'Japan',
     'describe': 'talk the journe to become a super ninja of Naruto',
     'genre': ['Animation', 'War'],
     'movie': 'Naruto',
     'star': 4,
     'year_release': '1998'},
    '_type': 'movie'},
   {'_id': '1',
    '_index': 'idmov',
    '_score': 0.5753642,
    '_source': {'auther': 'john',
     'country': 'USA, Canada',
     'describe': 'the love story of Linda and Jub',
     'genre': ['Animation', "Children's", 'Comedy'],
     'movie': 'Toy Story',
     'star': 2,
     'year_release': '1995'},
    '_type': 'movie'},
   {'_id': '2',
    '_index': 'idmov',
    '_score': 0.44079232,
    '_source': {'auther': 'kevin',
     'country': 'USA, England',
     'describe': 'the journey on the atlantic ocean of Titanic ship',
     'genre': ["Children's", 'drama'],
     'movie': 'Tita

In [57]:
a_3 = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "match" : {
            "describe" : {
                "query" : "the journe of",
                "zero_terms_query": "all"
            }
        }
    }
})
a_3

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'idmov',
    '_score': 1.0354949,
    '_source': {'auther': 'Kimito',
     'country': 'Japan',
     'describe': 'talk the journe to become a super ninja of Naruto',
     'genre': ['Animation', 'War'],
     'movie': 'Naruto',
     'star': 4,
     'year_release': '1998'},
    '_type': 'movie'},
   {'_id': '1',
    '_index': 'idmov',
    '_score': 0.5753642,
    '_source': {'auther': 'john',
     'country': 'USA, Canada',
     'describe': 'the love story of Linda and Jub',
     'genre': ['Animation', "Children's", 'Comedy'],
     'movie': 'Toy Story',
     'star': 2,
     'year_release': '1995'},
    '_type': 'movie'},
   {'_id': '2',
    '_index': 'idmov',
    '_score': 0.44079232,
    '_source': {'auther': 'kevin',
     'country': 'USA, England',
     'describe': 'the journey on the atlantic ocean of Titanic ship',
     'genre': ["Children's", 'drama'],
     'movie': 'Tita

# constant score query
+ a query wraps another query and simply returns a constant score equal to the query boost for every document in the filter
+ return the matches documents to have the score is equal

In [58]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "constant_score": {
            "filter":{
                "term" : {"describe": "the journe"}
            }
        }
    }
})
a['hits']['hits']

[]

In [59]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "constant_score": {
            "filter":{
                "term" : {"genre": "drama"}
            }
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 1.0,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

In [60]:
a = es.search(index = 'idmov', doc_type = 'movie', body = {
    "query": {
        "constant_score": {
            "filter":{
                "term" : {"genre": "drama"}
            },
            "boost": 1.2
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 1.2,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 1.2,
  '_source': {'auther': 'Max',
   'country': 'USA',
   'describe': 'the story talk about Tony stark, a super hero',
   'genre': ['drama', 'War'],
   'movie': 'Iron Man',
   'star': 3,
   'year_release': '2010'},
  '_type': 'movie'}]

# function_score
+ the function_score allows you modify the score of document that are retrieved by a query

In [61]:
a = es.search(index = 'idmov', doc_type= 'movie', body = {
    "query": {
        "function_score":{
            "query": {"match_all": {}},
            "boost": "5",
            "random_score": {},
            "boost_mode": "multiply"
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 4.5888333,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '4',
  '_index': 'idmov',
  '_score': 4.284919,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'},
 {'_id': '1',
  '_index': 'idmov',
  '_score': 3.6389053,
  '_source': {'auther': 'john',
   'country': 'USA, Canada',
   'describe': 'the love story of Linda and Jub',
   'genre': ['Animation', "Children's", 'Comedy'],
   'movie': 'Toy Story',
   'star': 2,
   'year_release': '1995'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 2.3637059,
  '_source': {'auther

In [62]:
a = es.search(index = 'idmov', doc_type= 'movie', body = {
    "query": {
        "function_score": {
          "query": { "match_all": {} },
          "boost": "5", 
          "functions": [
              {
                  "filter": { "match": { "describe": "the jounre" } },
                  "random_score": {}, 
                  "weight": 23
              },
              {
                  "filter": { "match": { "describe": "the story" } },
                  "weight": 42
              }
          ],
          "max_boost": 42,
          "score_mode": "max",
          "boost_mode": "multiply",
          "min_score" : 42
        }
    }
})
a['hits']['hits']

[{'_id': '2',
  '_index': 'idmov',
  '_score': 210.0,
  '_source': {'auther': 'kevin',
   'country': 'USA, England',
   'describe': 'the journey on the atlantic ocean of Titanic ship',
   'genre': ["Children's", 'drama'],
   'movie': 'Titanic',
   'star': 5,
   'year_release': '2000'},
  '_type': 'movie'},
 {'_id': '4',
  '_index': 'idmov',
  '_score': 210.0,
  '_source': {'auther': 'Kimito',
   'country': 'Japan',
   'describe': 'talk the journe to become a super ninja of Naruto',
   'genre': ['Animation', 'War'],
   'movie': 'Naruto',
   'star': 4,
   'year_release': '1998'},
  '_type': 'movie'},
 {'_id': '1',
  '_index': 'idmov',
  '_score': 210.0,
  '_source': {'auther': 'john',
   'country': 'USA, Canada',
   'describe': 'the love story of Linda and Jub',
   'genre': ['Animation', "Children's", 'Comedy'],
   'movie': 'Toy Story',
   'star': 2,
   'year_release': '1995'},
  '_type': 'movie'},
 {'_id': '3',
  '_index': 'idmov',
  '_score': 210.0,
  '_source': {'auther': 'Max',
   'c

# delete index

In [66]:
for i in range(1, len(e_dict)+1):
    res = es.delete(index = 'idmov', doc_type = 'movie', id = int(i))